In [1]:
# May help with conversion of bounding boxes to yolo format

# %%capture
# !pip install xmltodict
import numpy as np
import xmltodict
import pandas as pd
import re
import cv2
# from google.colab.patches import cv2_imshow
from PIL import Image
import os
import shutil
from pprint import pprint
from xml.etree import ElementTree as ET
# from google.colab import drive
# drive.mount('/content/drive')
from tqdm.notebook import tqdm

def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split("([0-9]+)", key)]
    return sorted(data, key=alphanum_key)

def pascal_voc_to_yolo(x1, y1, x2, y2, image_w, image_h):
    return [((x2 + x1)/(2*image_w)), ((y2 + y1)/(2*image_h)), (x2 - x1)/image_w, (y2 - y1)/image_h]

def coco_to_yolo(x1, y1, w, h, image_w, image_h):
    return [((2*x1 + w)/(2*image_w)) , ((2*y1 + h)/(2*image_h)), w/image_w, h/image_h]

def yolo_to_xml_bbox(bbox, w, h):
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2
    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)
    return xmin, ymin, xmax, ymax

In [ ]:
# XML pascal voc to Yolo

ORIGINAL_WIDTH = 1920
ORIGINAL_HEIGHT = 1080
FLOW_WIDTH = 1280
FLOW_HEIGHT = 720

annotations_dir = (
    "/content/drive/MyDrive/Scientific ISR/1_data/PESMOD/Pexels-Zaborski/annotations/"
)

output_dir = annotations_dir.replace("annotations", "YOLO_annotations_720_1280")
os.makedirs(output_dir, exist_ok=True)

annotations_name_list = sorted_alphanumeric(os.listdir(annotations_dir))

for annotation_name in tqdm(annotations_name_list):
    xml_path = annotations_dir + annotation_name
    k_width = ORIGINAL_WIDTH / FLOW_WIDTH
    k_height = ORIGINAL_HEIGHT / FLOW_HEIGHT
    with open(xml_path, "r") as file:
        filedata = file.read()
    data_dict = xmltodict.parse(filedata)
    try:
        object_data_dict_list = [dict(x) for x in data_dict["annotation"]["object"]]
        pascal_voc_bboxes = np.empty([len(object_data_dict_list), 4])
        yolo_bboxes = np.empty([len(object_data_dict_list), 5])
        for i, object_data_dict in enumerate(object_data_dict_list):
            xmin, ymin, xmax, ymax = (
                np.ceil(float(object_data_dict["bndbox"]["xmin"]) / k_width),
                np.ceil(float(object_data_dict["bndbox"]["ymin"]) / k_height),
                np.ceil(float(object_data_dict["bndbox"]["xmax"]) / k_width),
                np.ceil(float(object_data_dict["bndbox"]["ymax"]) / k_height),
            )
            pascal_voc_bboxes[i] = xmin, ymin, xmax, ymax
            yolo_bboxes[i] = np.append(
                0, pascal_voc_to_yolo(xmin, ymin, xmax, ymax, FLOW_WIDTH, FLOW_HEIGHT)
            )
            output_path = output_dir + annotation_name.replace("xml", "txt")
            result = np.savetxt(
                output_path, yolo_bboxes, fmt=["%d", "%.6f", "%.6f", "%.6f", "%.6f"]
            )
    except ValueError:
        object_data_dict = data_dict["annotation"]["object"]
        xmin, ymin, xmax, ymax = (
            np.ceil(float(object_data_dict["bndbox"]["xmin"]) / k_width),
            np.ceil(float(object_data_dict["bndbox"]["ymin"]) / k_height),
            np.ceil(float(object_data_dict["bndbox"]["xmax"]) / k_width),
            np.ceil(float(object_data_dict["bndbox"]["ymax"]) / k_height),
        )
        pascal_voc_bboxes = xmin, ymin, xmax, ymax
        yolo_bboxes = np.append(
            0, pascal_voc_to_yolo(xmin, ymin, xmax, ymax, FLOW_WIDTH, FLOW_HEIGHT)
        )
        yolo_bboxes = np.reshape(yolo_bboxes, (1, 5))
        output_path = output_dir + annotation_name.replace("xml", "txt")
        result = np.savetxt(output_path, yolo_bboxes, fmt="%d %.6f %.6f %.6f %.6f")
    except KeyError:
        result = np.savetxt(output_path, np.array([]))
        print("Empty file!")
# source = '/content/drive/MyDrive/Scientific ISR/1_data/PESMOD/Pexels-Zaborski/images'
# target = '/content/drive/MyDrive/Scientific ISR/Detection/train_data/images/train'

# files=os.listdir(source)

# for fname in tqdm(files):
#     shutil.copy2(os.path.join(source,fname), target)


In [ ]:
# Coco to yolo

SEQUENCE_DIR = "C:/Users/pv5/Desktop/flow2fix/"
ANNO_DIR = "C:/Users/pv5/Desktop/annos2fix/"
ANNO_YOLO_DIR = "C:/Users/pv5/Desktop/fixed_annos/"

forbidden_sequence_name_list = [
    "bird1",
    "car1",
    "car6",
    "car8",
    "car16",
    "group1",
    "group2",
    "group3",
    "person2",
    "person4",
    "person5",
    "person7",
    "person8",
    "person12",
    "person14",
    "person17",
    "person19",
    "truck4",
    "uav1",
]

allowed_sequence_name_list = ["car2", "car7", "car12", "car14", "person9"]

all_sequence_names = sorted_alphanumeric(os.listdir(SEQUENCE_DIR))
for i in tqdm(range(len(all_sequence_names))):
    current_sequence_name = all_sequence_names[i]
    if (
        current_sequence_name not in forbidden_sequence_name_list
        and current_sequence_name in allowed_sequence_name_list
    ):
        current_sequence_bboxes = np.loadtxt(
            ANNO_DIR + current_sequence_name + ".txt", delimiter=","
        )
        current_sequence_img_dir = SEQUENCE_DIR + current_sequence_name + "/"
        current_sequence_img_names_list = sorted_alphanumeric(
            os.listdir(current_sequence_img_dir)
        )
        os.makedirs(ANNO_YOLO_DIR + current_sequence_name, exist_ok=True)
        for j in tqdm(range(len(current_sequence_bboxes))):
            current_frame_height, current_frame_width, _ = cv2.imread(
                current_sequence_img_dir + current_sequence_img_names_list[j]
            ).shape
            current_frame_bboxes = current_sequence_bboxes[j]
            try:
                x_min = int(current_frame_bboxes[0])
                y_min = int(current_frame_bboxes[1])
                object_width = int(current_frame_bboxes[2])
                object_height = int(current_frame_bboxes[3])
                yolo_bboxes = np.append(
                    0,
                    coco_to_yolo(
                        x_min,
                        y_min,
                        object_width,
                        object_height,
                        current_frame_width,
                        current_frame_height,
                    ),
                )
                yolo_bboxes = np.reshape(yolo_bboxes, (1, 5))
                output_path = (
                    ANNO_YOLO_DIR
                    + current_sequence_name
                    + "/"
                    + current_sequence_img_names_list[j].replace("jpg", "txt")
                )
                result = np.savetxt(
                    output_path, yolo_bboxes, fmt="%d %.6f %.6f %.6f %.6f"
                )
            except ValueError:
                print("Empty file!")
                output_path = (
                    ANNO_YOLO_DIR
                    + current_sequence_name
                    + "/"
                    + current_sequence_img_names_list[j].replace("jpg", "txt")
                )
                result = np.savetxt(output_path, np.array([np.nan]))
    else:
        print("Forbidden sequence!")


In [ ]:
images_dir = "C:/Users/data/images/"
labels_dir = "C:/Users/data/labels/"

images_dst = "C:/Users/train_data/images/train/"
labels_dst = "C:/Users/train_data/labels/train/"

img_names = np.array((sorted_alphanumeric(os.listdir(images_dir))))
label_names  = np.array((sorted_alphanumeric(os.listdir(labels_dir))))

train_indices = (np.random.choice(img_names.shape[0], int(len(img_names) * 0.80), replace=False))

for i in tqdm(range(len(train_indices))):
    img_path = images_dir + img_names[train_indices[i]]
    label_path = labels_dir + label_names[train_indices[i]]
    shutil.move(img_path, images_dst+ img_names[train_indices[i]])
    shutil.move(label_path, labels_dst+ label_names[train_indices[i]])
    
print(img_path)
print(label_path)